In [1]:
a=[]
a.write

AttributeError: 'list' object has no attribute 'write'

In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [25]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.delete_all_components()

In [4]:
design.get_chip_z('Q_chip')

'20um'

In [3]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='20um'

# Resonator length

In [5]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [6]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.35
    resonator_length_2.append(a)
print(resonator_length_2)


[8.103254683870285, 7.874788339841421, 7.658346540209131]


In [12]:
resonator_length_4=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,4)
    a=a.strip('mm')
    a=float(a)
    resonator_length_4.append(a)
print(resonator_length_4)


[4.226627341935142, 4.1123941699207105, 4.004173270104565]


# Flipchip

In [26]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
cp1 = dict(down = dict(loc_W=-1, loc_H=-1, pad_width='120um'))

q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip',
                                            connection_pads=cp0))]
q1=[TransmonPocket6(design, 'Q1', options=dict(pos_x='2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip',
                                            connection_pads=cp1))]
qubits=q0+q1

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))
q1x=float(qubits[1].options.pos_x.strip('um'))
q1y=float(qubits[1].options.pos_y.strip('um'))

c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-100}um",
                                              pos_y=f"{q0y+100}um",
                                              pad_width="150um",
                                              pad_height="80um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))
c1 = RightCoupler(design, "c1", options=dict(pos_x=f"{q1x+100}um",
                                              pos_y=f"{q1y+100}um",
                                              pad_width="150um",
                                              pad_height="80um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))  

design.rebuild()
gui.rebuild()
gui.autoscale()

In [26]:
#planar랑 똑같은 디자인
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
cp1 = dict(down = dict(loc_W=-1, loc_H=-1, pad_width='120um'))

q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip',
                                            connection_pads=cp0))]
q1=[TransmonPocket6(design, 'Q1', options=dict(pos_x='2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip',
                                            connection_pads=cp1))]
qubits=q0+q1

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))
q1x=float(qubits[1].options.pos_x.strip('um'))
q1y=float(qubits[1].options.pos_y.strip('um'))

c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-150}um",
                                              pos_y=f"{q0y+140}um",
                                              pad_width="150um",
                                              pad_height="30um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))
c1 = RightCoupler(design, "c1", options=dict(pos_x=f"{q1x+150}um",
                                              pos_y=f"{q1y+140}um",
                                              pad_width="150um",
                                              pad_height="30um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))  

design.rebuild()
gui.rebuild()
gui.autoscale()

# Launchpad

In [27]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

# Feedline

In [28]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    mirror=True,
                                    open_termination=True))
clt1 = CoupledLineTee(design, "clt1", 
                        options=dict(pos_x="2000um",
                                    pos_y='2000um',
                                    coupling_length='300um', 
                                    chip="C_chip",
                                    down_length='300um',
                                    mirror=False,
                                    open_termination=True))


In [29]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='clt1',
                                                            pin='prime_start'))))
fl2=RouteStraight(design, 'fl2',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt1',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))

# Readout Resonator

In [11]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [30]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')
rr1=connect('rr1', 'clt1', 'second_end', 'c1', 'in',
            resonator_length_2[0],jogs,jogs, 
            '400um','0um', '100um', '100um')
bus=connect('bus', 'Q0', 'down', 'Q1', 'down',
            resonator_length_2[2],jogs,jogs, 
            '300um','-800um', '100um', '100um')            

# LOM Analysis

In [47]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [48]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.18',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.18',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [2]:
c1.sim.close()

NameError: name 'c1' is not defined

In [50]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('Q0', 'down'), ('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 05:08PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmppbr4x09m.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5180000000, Maxwell, 1, False
INFO 05:08PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpt1eg7b8f.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5180000000, Maxwell, 1, False
INFO 05:08PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp4mwe9vlm.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5180000000, Maxwell, 2, False
INFO 05:08PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp7lmol42x.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5180000000, Maxwell, 3, False
INFO 05:08PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpb6uqml6u.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5180000000, Maxwell, 4, False
INFO 05:08

,down_connector_pad_Q0,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
down_connector_pad_Q0,48.08500,-34.801,-11.62200,-1.2957,-0.00505
ground,-34.80100,310.000,-68.61300,-73.3590,-67.10800
pad_bot_Q0,-11.62200,-68.613,112.55000,-30.9850,-0.14186
pad_top_Q0,-1.29570,-73.359,-30.98500,107.8700,-1.17190
quad_coupler_c0,-0.00505,-67.108,-0.14186,-1.1719,68.57500


In [51]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.72
c1.setup.freq_bus = [7.99]
c1.run_lom()
c1.lumped_oscillator_all

[2, 3] [4 0]
Predicted Values

Transmon Properties
f_Q 5.371451 [GHz]
EC 269.020755 [MHz]
EJ 14.854145 [GHz]
alpha -306.408154 [MHz]
dispersion 4.370808 [KHz]
Lq 10.995564 [nH]
Cq 72.002727 [fF]
T1 2463.558253 [us]

**Coupling Properties**

tCqbus1 -0.534155 [fF]
gbus1_in_MHz -7.314068 [MHz]
χ_bus1 -0.014919 [MHz]
1/T1bus1 15.604141 [Hz]
T1bus1 10199.532661 [us]

tCqbus2 4.975001 [fF]
gbus2_in_MHz 80.652055 [MHz]
χ_bus2 -0.543304 [MHz]
1/T1bus2 48.999545 [Hz]
T1bus2 3248.090226 [us]
Bus-Bus Couplings
gbus1_2 0.380736 [MHz]


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.861741,323.917781,14.854145,-374.97068,26.438696,"[-46.10846407629163, 60.565399522744286]","[-1.516731429903507, -0.5310945240726785]",1.516731,46.108464
2,5.608167,294.803893,14.854145,-338.3674,10.847063,"[-4.09078012666261, 68.48271836211634]","[-0.00710027681145876, -0.5074580548337988]",0.007100,4.090780
3,5.541824,287.44304,14.854145,-329.19979,8.474872,"[-6.031339914524494, 75.49233860612307]","[-0.013649757697824198, -0.5728489261410972]",0.013650,6.031340
4,5.462899,278.823342,14.854145,-318.50861,6.268782,"[-6.5873446731526295, 76.64228283574771]","[-0.014146995725981661, -0.5415205246609784]",0.014147,6.587345
5,5.41821,274.008295,14.854145,-312.557157,5.264007,"[-6.745942842780424, 76.42761803112862]","[-0.013734702766371745, -0.5130354197963423]",0.013735,6.745943
6,5.397667,271.810765,14.854145,-309.845919,4.853037,"[-6.9248736475536585, 79.01294853329914]","[-0.01397632989927284, -0.5363270000559701]",0.013976,6.924874
7,5.381548,270.093409,14.854145,-307.729256,4.551146,"[-7.2972334906860326, 80.11733431620839]","[-0.015103866485774884, -0.5419583167060877]",0.015104,7.297233
8,5.371451,269.020755,14.854145,-306.408154,4.370808,"[-7.314067807935301, 80.65205488603584]","[-0.014919137203851325, -0.5433044616493049]",0.014919,7.314068


In [53]:
#c1.sim.run(components=['Q1','c1'], open_terminations=[('Q1', 'down'), ('c1', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 05:13PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpgcx22vrf.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5330000000, Maxwell, 1, False
INFO 05:13PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpagld5w2h.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5330000000, Maxwell, 1, False
INFO 05:13PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpy2gv3szf.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5330000000, Maxwell, 2, False
INFO 05:13PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmps6ci3vog.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5330000000, Maxwell, 3, False
INFO 05:13PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp7jivd2i7.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5330000000, Maxwell, 4, False
INFO 05:13

,down_connector_pad_Q1,ground,pad_bot_Q1,pad_top_Q1,quad_coupler_c1
down_connector_pad_Q1,47.325000,-34.238,-11.44600,-1.2874,-0.005457
ground,-34.238000,307.580,-68.70600,-73.4020,-64.855000
pad_bot_Q1,-11.446000,-68.706,112.14000,-30.7150,-0.144640
pad_top_Q1,-1.287400,-73.402,-30.71500,107.5900,-1.155700
quad_coupler_c1,-0.005457,-64.855,-0.14464,-1.1557,66.304000


In [54]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.87
c1.setup.freq_bus = [7.99]
c1.run_lom()
c1.lumped_oscillator_all

[2, 3] [4 0]
Predicted Values

Transmon Properties
f_Q 5.381480 [GHz]
EC 270.086131 [MHz]
EJ 14.854145 [GHz]
alpha -307.720289 [MHz]
dispersion 4.549902 [KHz]
Lq 10.995564 [nH]
Cq 71.718707 [fF]
T1 2599.600147 [us]

**Coupling Properties**

tCqbus1 -0.524066 [fF]
gbus1_in_MHz -7.358669 [MHz]
χ_bus1 -0.012692 [MHz]
1/T1bus1 12.957821 [Hz]
T1bus1 12282.539408 [us]

tCqbus2 4.897788 [fF]
gbus2_in_MHz 79.649845 [MHz]
χ_bus2 -0.535613 [MHz]
1/T1bus2 48.265034 [Hz]
T1bus2 3297.520555 [us]
Bus-Bus Couplings
gbus1_2 0.381806 [MHz]


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.826326,319.757554,14.854145,-369.706534,23.456225,"[-46.79202117246007, 58.47954595562012]","[-1.107849286540532, -0.47493140282945534]",1.107849,46.792021
2,5.580515,291.723018,14.854145,-334.526066,9.793865,"[-3.3949799167964048, 66.96439628570158]","[-0.003733503745169711, -0.47047511443142137]",0.003734,3.394980
3,5.524244,285.510167,14.854145,-326.798261,7.930371,"[-5.849907670121121, 72.86715537688985]","[-0.01008667461391549, -0.5234619703281337]",0.010087,5.849908
4,5.47254,279.868279,14.854145,-319.802126,6.507053,"[-5.874074576937013, 73.03408668901592]","[-0.009345047868112575, -0.496921669356587]",0.009345,5.874075
5,5.415987,273.769951,14.854145,-312.262946,5.218052,"[-6.49551897016879, 75.96837312828715]","[-0.010439207021360974, -0.505673444954224]",0.010439,6.495519
6,5.393969,271.416161,14.854145,-309.359397,4.782207,"[-7.02155941323818, 78.84242484019722]","[-0.01178334519309352, -0.5318947390416744]",0.011783,7.021559
7,5.38148,270.086131,14.854145,-307.720289,4.549902,"[-7.358668901615214, 79.64984528316273]","[-0.012691974289008746, -0.5356127324308688]",0.012692,7.358669


# EPR Analysis

In [31]:
from qiskit_metal.analyses.quantization import EPRanalysis

In [32]:
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [33]:
hfss.start()

INFO 02:47PM [connect_project]: Connecting to Ansys Desktop API...
INFO 02:47PM [load_ansys_project]: 	Opened Ansys App
INFO 02:47PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 02:47PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project21
INFO 02:47PM [connect_design]: 	Opened active design
	Design:    Flip_chip [Solution type: Eigenmode]
INFO 02:47PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 02:47PM [connect]: 	Connected to project "Project21" and design "Flip_chip" 😀 



True

In [35]:
hfss.activate_ansys_design("Flipchip", 'eigenmode')
hfss.render_design(['Q0','Q1','bus','c0','c1','lpL','lpR','clt0','clt1','rr0','rr1','fl0','fl1','fl2'],
                    [('clt0','prime_end'),('clt1','prime_start'),
                    ('bus','start'),('bus','end')],
                    box_plus_buffer=True)

INFO 02:49PM [connect_design]: 	Opened active design
	Design:    Flipchip [Solution type: Eigenmode]


In [1]:
hfss.close()

NameError: name 'hfss' is not defined